# Rayleigh-Bénard Convection
Rayleigh-Bénard Convection is the flow induced in a fluid which is heated from below and resulting in convective transport of engergy from the heated lower to the cooler upper boundary. It is implemented here in two dimensions with periodic boundary conditions. The subtlety of the problem is incompressebility: a pressure gradient influences the velocity field, which has to be chosen such that the resulting flow is obeying incompressebility.

In [ ]:
import jax.numpy as jnp
from jax import jit
from jax import tree_util
from jax.scipy.signal import convolve2d

import scipy.sparse as sp
from scipy.linalg import inv
from scipy import ndimage
import numpy as np
from matplotlib import pyplot as plt

from adoptODE import dataset_adoptODE, train_adoptODE, simple_simulation
from adoptODE import sgd_bounded
from adoptODE.ODE_Fix_dt import odeint

In [ ]:
def define_RaBe2d_per(**kwargs_sys):
    disc_x, disc_z = kwargs_sys['disc_x'], kwargs_sys['disc_z']
    dx, dz = kwargs_sys['dx'], kwargs_sys['dz']
    N_sys =  kwargs_sys['N_sys']
    # dt_est = 1e-4

    def d_dx(v):
        return (jnp.roll(v, -1, axis=0)-jnp.roll(v, 1, axis=0))/(2*dx)
    def d_dz(v_ext): # expects extendet inputs
        return (v_ext[:,2:] - v_ext[:,:-2])/(2*dz)
    def gradient(v_ext): #gradient of scalar field
        gx = d_dx(v_ext[:,1:-1])
        gz = d_dz(v_ext)
        return jnp.stack((gx,gz), axis=2)
    
    stencil5 = np.array([1/4,1/2,-1.5,1/2,1/4])
    distr5 = np.array([0.05,0.1,0.7,0.1,0.05])
    kernel = np.outer(distr5, stencil5/dz**2) + np.outer(stencil5/dx**2, distr5)

    wrap_padding = 2
    def laplace2(v): #laplace of scalar
        v_ext = jnp.concatenate((v[-wrap_padding:], v, v[:wrap_padding]), axis=0)
        return convolve2d(v_ext, kernel, mode='same')[wrap_padding:-wrap_padding]
    def laplace2v(v): #laplace of vector
        v_ext = jnp.concatenate((v[-wrap_padding:], v, v[:wrap_padding]), axis=0)
        return jnp.stack((convolve2d(v_ext[...,0], kernel, mode='same'), convolve2d(v_ext[...,1], kernel, mode='same')), axis=2)[wrap_padding:-wrap_padding]
    def divergence(v_ext):
        vx = d_dx(v_ext[:,1:-1,0])
        vz = d_dz(v_ext[...,1])
        return vx+vz
    def vector_divergence_vect(v_ext1, v_ext2): #v_ext2 is vector
        v1 = v_ext1[:,1:-1]
        v2 = v_ext2[:,1:-1]
        return jnp.einsum('ij,ijk->ijk',v1[:,:,0],d_dx(v2)) + jnp.einsum('ij,ijk->ijk',v1[:,:,1],d_dz(v_ext2))
    def vector_divergence_scalar(v_ext1, v_ext2): #v_ext2 is scalar
        v1 = v_ext1[:,1:-1]
        v2 = v_ext2[:,1:-1]
        return v1[:,:,0]*d_dx(v2) + v1[:,:,1]*d_dz(v_ext2)
    
    def Diff_mat_1D(Nx):
        # First derivative
        D_1d = sp.diags([-1, 1], [-1, 1], shape = (Nx,Nx)) # A division by (2*dx) is required later.
        D_1d = sp.lil_matrix(D_1d)
        D_1d[0,[0,1,2]] = [-3, 4, -1]               # this is 2nd order forward difference (2*dx division is required)
        D_1d[Nx-1,[Nx-3, Nx-2, Nx-1]] = [1, -4, 3]  # this is 2nd order backward difference (2*dx division is required)
        
        # # Second derivative
        corr = 1e-6
        lap_kern = [1-corr, 4*corr, -2 - 6*corr, 4*corr, 1-corr] #This correction is neccessary as otherwise only ever other colum is connected via the laplace operator.
        fw_lap_kern = np.array([2, -5, 4, -1])*4
        fw2_lap_kern = np.array([3, -5, 1,1]) # (-1,0,1) - Forward on edge 
        # Second derivative
        D2_1d =  sp.diags(lap_kern, [-2,-1,0,1,2], shape = (Nx, Nx)) # division by dx^2 required
        D2_1d = sp.lil_matrix(D2_1d)                  
        D2_1d[0,[0,1,2,3]] = fw_lap_kern #[-1, 0, 1, 0]                    # this is 2nd order forward difference. division by dx^2 required. 
        D2_1d[1,[0,1,2,3]] = fw2_lap_kern #[2,-3,0,1]                  # this is 2nd order forward difference. division by dx^2 required. #CHANGED TO SUM TO ZERO
        D2_1d[Nx-2,[Nx-4, Nx-3, Nx-2, Nx-1]] = fw2_lap_kern[::-1] #[1,0,-3,2]  # this is 2nd order backward difference. division by dx^2 required.
        D2_1d[Nx-1,[Nx-4, Nx-3, Nx-2, Nx-1]] = fw_lap_kern[::-1] #[0, 1, 0, -1]  # this is 2nd order backward difference. division by dx^2 required.

        # Second periodic
        D2_1dp =  sp.diags(lap_kern, [-2,-1,0,1,2], shape = (Nx, Nx)) # division by dx^2 required
        D2_1dp = sp.lil_matrix(D2_1dp)                  
        D2_1dp[0,[Nx-2,Nx-1,0,1,2]] = lap_kern                   # this is 2nd order forward difference. division by dx^2 required. 
        D2_1dp[1,[Nx-1,0,1,2,3]] = lap_kern                 # this is 2nd order forward difference. division by dx^2 required. 
        D2_1dp[Nx-2,[Nx-4, Nx-3, Nx-2, Nx-1, 0]] = lap_kern  # this is 2nd order backward difference. division by dx^2 required.
        D2_1dp[Nx-1,[Nx-3, Nx-2, Nx-1, 0, 1]] = lap_kern  # this is 2nd order backward difference. division by dx^2 required.
        
        return D_1d, D2_1d/4, D2_1dp/4 # /4 in order for later only divide by dx**2 instead 4x**2

    def Diff_mat_2D(Nx,Ny):
        # 1D differentiation matrices
        Dx_1d, _, D2x_1d = Diff_mat_1D(Nx)
        Dy_1d, D2y_1d, _ = Diff_mat_1D(Ny)
        # Sparse identity matrices
        Ix = sp.eye(Nx)
        Iy = sp.eye(Ny)
        # 2D matrix operators from 1D operators using kronecker product
        # First partial derivatives
        Dx_2d = sp.kron(Dx_1d,Iy)
        Dy_2d = sp.kron(Ix,Dy_1d)
        # Second partial derivatives
        D2x_2d = sp.kron(D2x_1d,Iy)
        D2y_2d = sp.kron(Ix,D2y_1d)
        return (Dx_2d.toarray()), (Dy_2d.toarray()), (D2x_2d.toarray()), (D2y_2d.toarray())

    def build_laplace():
        Nx = disc_x
        Nz = disc_z+2
        Dx_2d, Dy_2d, D2x_2d, D2y_2d = Diff_mat_2D(Nx,Nz)
        laplace_op = (D2x_2d/dx**2+D2y_2d/dz**2)
        id_boundary = np.eye(Nx*Nz)
        gridvals_x = np.linspace(0,1,Nx)
        gridvals_z = np.linspace(0,1,Nz)
        X,Y = np.meshgrid(gridvals_x, gridvals_z, indexing='ij')   
        Xu = X.ravel()          
        Yu = Y.ravel()
        Bottom = np.where(Yu==0)[0]
        Top = np.where(Yu==1)[0]
        SngPoint = np.where((Yu==gridvals_z[0])*(Xu==gridvals_x[20]))[0]
        laplace_op = np.array(laplace_op)
        laplace_op[Top,:]=Dy_2d[Top,:]/(2*dz)
        laplace_op[Bottom,:]=Dy_2d[Bottom,:]/(2*dz)
        laplace_op[SngPoint,:] = id_boundary[SngPoint,:]*1

        return laplace_op

    def build_inv_laplace():
        return inv(build_laplace())

    @jit
    def solve_lap(div):
        return jnp.matmul(build_inv_laplace(), div.flatten()).reshape((disc_x,disc_z+2))

    @jit
    def f_comps(y, t, dt, params, iparams, exparams): #A pseudop-pressure method to enforce toroidal flow is used (Rempfer, 2006, DOI: 10.1115/1.2177683)
        u = y['velocity'] #shape x . y . z . v
        T = y['temperature_p']

        Ra = 10**params['RaExp']
        Pr = params['Pr']

        u_ext = jnp.concatenate((jnp.zeros((disc_x, 1, 2)), u, jnp.zeros((disc_x, 1, 2))), axis=1) # no_slip boundary for velocity, 2 extendet to get force at boundary
        u_ext2 = jnp.concatenate((jnp.zeros((disc_x, 2, 2)), u, jnp.zeros((disc_x, 2, 2))), axis=1) # no_slip boundary for velocity, 2 extendet to get force at boundary
        T_ext = jnp.concatenate((jnp.zeros((disc_x, 1)), T, jnp.zeros((disc_x, 1))), axis=1) # zero temperature perturbation at boundaries

        dTdt = - vector_divergence_scalar(u_ext, T_ext) + u[...,1] + laplace2(T)
        dudt_wo_p = Pr * Ra * jnp.tensordot(T, jnp.array([0,1]), axes=((),())) + Pr * laplace2v(u) - vector_divergence_vect(u_ext, u_ext)
        dudt_wo_p_ext = jnp.concatenate((jnp.zeros((disc_x, 1, 2)), dudt_wo_p, jnp.zeros((disc_x, 1, 2))), axis=1)

        dt_mod = 10*dt
        
        div = divergence(dudt_wo_p_ext*dt_mod + u_ext)
        div = jnp.concatenate((jnp.zeros((disc_x,1)), div, jnp.zeros((disc_x,1))), axis=1)

        p = solve_lap(div)/dt_mod

        dudt = dudt_wo_p - gradient(p)

        return dTdt, dudt_wo_p_ext, p, dudt

    @jit
    def f(y, t, dt, params, iparams, exparams):
        dTdt, dudt_wo_p_ext, p, dudt = f_comps(y, t, dt, params, iparams, exparams)

        return {'velocity':dudt, 'temperature_p':dTdt}
    @jit
    def return_forces(y, t, dt, params, iparams, exparams):
        dTdt, dudt_wo_p_ext, p, dudt = f_comps(y, t, dt, params, iparams, exparams)

        u = y['velocity']
        u_ext = jnp.concatenate((jnp.zeros((disc_x, 1, 2)), u, jnp.zeros((disc_x, 1, 2))), axis=1) # no_slip boundary for velocity, 2 extendet to get force at boundary
        return {'dTdt':dTdt, 'dudt_wo_p':dudt_wo_p_ext, 'div_dudt_wo_p':divergence(dudt_wo_p_ext), 'p':p, 'grad_p':gradient(p), 'dudt':dudt, 'div_dudt_wo_p_ext':divergence(dudt_wo_p_ext), 'div_dudt':divergence(dudt), 'div_u':divergence(u_ext)}
    
    if 'lap_loss' in kwargs_sys:
        lap_loss = kwargs_sys['lap_loss']
        if 't_for_lap_loss' in kwargs_sys:
            t_idx = kwargs_sys['t_for_lap_loss']
        else:
            t_idx = 0
    else:
        lap_loss = 0
    
    
    @jit
    def loss(ys, params, iparams, exparams, targets):
        u = ys['velocity']
        u_target = targets['velocity']
        return jnp.nanmean((u-u_target)**2)


    def gen_params():
        Pr = np.random.rand()*3.7+0.3
        RaExp = 4*np.random.rand()+2
        return {'Pr':Pr, 'RaExp':RaExp}, {}, {}


    def gen_y0():
        return {'velocity':np.zeros((disc_x, disc_z, 2)), 'temperature_p':1*ndimage.gaussian_filter(np.random.randn(disc_x, disc_z), disc_x//7,mode='wrap')}#0.01*np.random.randn(disc_x, disc_z)}#  

    return f, loss, gen_params, gen_y0, {'return_forces':jit(return_forces)}

### Temperature Reconstruction in a smaller system
Data is simulated and afterwards the goal is to reconstruct the temperature field. 101 time steps are simulated, with the first large time step is ignored for training (Convection is only starting to build up at the beginning).

In [ ]:
# Setting up Simulation and Training Parameters
# 101 timesteps are simulated, where the first timestep is choosen longer and ignored for training, such that the initial transient period is skipped
N_times = 50
t_evals=np.insert(np.linspace(0,0.005,N_times)+0.02,0,0)

res=50 # Pixel resolution of the simulation domain
kwargs_sys = {'disc_x':int(1*res), 'disc_z':res, 'dx':1/res, 'dz':1/res, 't_evals':t_evals, 'N_sys':1}

# The following use of the t_reset_idcs keyword with a single entry leads to training starting only with the trajectory from that time
start_rec_idx = 1
t_reset_idcs= (start_rec_idx,) 
params_train = {'Pr':2.0, 'RaExp':6.0}
kwargs_adoptODE = {'epochs':100, 'lr':0, 'lr_y0':0.05, 'N_backups':10,
               'atol':1e-5, 'rtol':1e-5, 'EOM_dt_dependent':True, 'ODE_solver':odeint,
              't_reset_idcs':t_reset_idcs}#,
              #'optimizer_y0':sgd_bounded, 'optimizer_y0_kwargs':{}} # For some parameters stochastic gradient decent might be favorable.
dataset_sim = simple_simulation(define_RaBe2d_per, t_evals, kwargs_sys, kwargs_adoptODE, params = params_train, params_train=params_train)

In [ ]:
# Plot initial temperature field
fig, ax = plt.subplots(1,1)
ax.imshow(dataset_sim.ys['temperature_p'][0,start_rec_idx].T, origin='lower')
ax.set_axis_off()

In [ ]:
# Forgetting the temperature field + Setting boundaries such that the velocity field is not changed
temperature_truth = dataset_sim.ys['temperature_p'][0].copy() # Save true temperature field for later comparison
dataset_sim.ys['temperature_p'] = dataset_sim.ys['temperature_p']*jnp.nan
dataset_sim.y0_train['temperature_p'] = np.zeros_like(dataset_sim.y0_train['temperature_p'])
lower_b = {'velocity':dataset_sim.y0_train['velocity'], 'temperature_p':-jnp.inf}
upper_b = {'velocity':dataset_sim.y0_train['velocity'], 'temperature_p':+jnp.inf}
dataset_sim.kwargs_adoptODE['lower_b_y0']=lower_b
dataset_sim.kwargs_adoptODE['upper_b_y0']=upper_b

In [ ]:
# First run, using 10 time steps, starting at the previously defined start_rec_idx
dataset_sim.kwargs_adoptODE['t_stop_idx'] = start_rec_idx+10
params_final, losses, errors, params_history = train_adoptODE(dataset_sim, print_interval=5)

In [ ]:
# Plot true vs recovered initial temperature field:
fig, ax = plt.subplots(1,2)
ax[0].imshow(temperature_truth[start_rec_idx].T, origin='lower')
ax[0].set_axis_off()
ax[0].set_title('True Temp. Perturbation')
ax[1].imshow(dataset_sim.ys_sol['temperature_p'][0,start_rec_idx].T, origin='lower')
ax[1].set_axis_off()
ax[1].set_title('Rec. Temp. Perturbation')

### Recovery at higher Resolution, Prandtl and Rayleigh number (Computationally Expensive!)

In [ ]:
# The pseudo-pressure correction for divergence free flow might interfere with the adaptive stepsize of the standard ode solver, hence we use a fixed timestep ODE solver here.
from adoptODE.ODE_Fix_dt import odeint as odeint_fixed

In [ ]:
# Setting up Simulation and Training Parameters
# 11 timesteps are simulated, where the first timestep is choosen longer and ignored for training, such that the initial transient period is skipped.
# This transient time also makes generating the initial conditions significantly time consuming for these parameters.
N_times = 10
t_evals=np.insert(np.linspace(0,0.0002,N_times)+0.01,0,0)

res=100 # Pixel resolution of the simulation domain
kwargs_sys = {'disc_x':int(1*res), 'disc_z':res, 'dx':1/res, 'dz':1/res, 't_evals':t_evals, 'N_sys':1}

# The following use of the t_reset_idcs keyword with a single entry leads to training starting only with the trajectory from that time
start_rec_idx = 1
t_reset_idcs= (start_rec_idx,) 
params_train = {'Pr':7.0, 'RaExp':7.0}
kwargs_adoptODE = {'epochs':50, 'lr':0, 'lr_y0':0.002, 'N_backups':15,
               'atol':1e-5, 'rtol':1e-5, 'EOM_dt_dependent':True, 'ODE_solver':odeint_fixed, 'dt':1e-7,
              't_reset_idcs':t_reset_idcs}
dataset_sim = simple_simulation(define_RaBe2d_per, t_evals, kwargs_sys, kwargs_adoptODE, params = params_train, params_train=params_train)

In [ ]:
# Plot initial temperature field
fig, ax = plt.subplots(1,1)
ax.imshow(dataset_sim.ys['temperature_p'][0,start_rec_idx].T, origin='lower')
ax.set_axis_off()

In [ ]:
# Forgetting the temperature field + Setting boundaries such that the velocity field is not changed
temperature_truth = dataset_sim.ys['temperature_p'][0].copy() # Save true temperature field for later comparison
dataset_sim.ys['temperature_p'] = dataset_sim.ys['temperature_p']*jnp.nan
dataset_sim.y0_train['temperature_p'] = np.zeros_like(dataset_sim.y0_train['temperature_p'])
lower_b = {'velocity':dataset_sim.y0_train['velocity'], 'temperature_p':-jnp.inf}
upper_b = {'velocity':dataset_sim.y0_train['velocity'], 'temperature_p':+jnp.inf}
dataset_sim.kwargs_adoptODE['lower_b_y0']=lower_b
dataset_sim.kwargs_adoptODE['upper_b_y0']=upper_b

In [ ]:
# Perform training: These parameters should yield accurate results, but will take hours to days depending on the setup.
# For a quick glance, one might try to reduce the number of epochs and increase the learning rate, though this might lead to NaN gradient errors.
dataset_sim.kwargs_adoptODE['epochs'] = 1000
dataset_sim.kwargs_adoptODE['t_stop_idx'] = 10
dataset_sim.kwargs_adoptODE['lr_y0'] = 0.003#0.0002 for sdg
res2 = train_adoptODE(dataset_sim, print_interval=10)

In [ ]:
# Plot true vs recovered initial temperature field:
fig, ax = plt.subplots(1,2)
ax[0].imshow(temperature_truth[start_rec_idx].T, origin='lower')
ax[0].set_axis_off()
ax[0].set_title('True Temp. Perturbation')
ax[1].imshow(dataset_sim.ys_sol['temperature_p'][0,start_rec_idx].T, origin='lower')
ax[1].set_axis_off()
ax[1].set_title('Rec. Temp. Perturbation')